In [1]:
include("../../src/WaterLily.jl")
WaterLily = Main.WaterLily;
using Plots; gr()
using StaticArrays
using JLD
using Images
using FFTW
using Statistics
using Interpolations
using DelimitedFiles
using LinearAlgebra
using PyPlot
using GLMakie
GLMakie.activate!()

In [2]:
Lp = 256
N = (Lp, Lp, Lp)

(256, 256, 256)

In [3]:
sim = WaterLily.Simulation(N, (0, 0, 0), Lp; U=1, ν=0.0,T=Float32,mem=Array,perdir=(1,2,3), Δt=0.00001)

xList = reshape((1:N[1]+2).-2,(N[1]+2,1,1))/N[1]
yList = reshape((1:N[2]+2).-2,(1,N[2]+2,1))/N[2]
zList = reshape((1:N[3]+2).-2,(1,1,N[3]+2))/N[3]


function leVeque!(u,x,y,z)
    u[:,:,:,1] = @. 2*sin( pi*x)^2*sin(2pi*y)  *sin(2pi*z)  *N[1]
    u[:,:,:,2] = @.  -sin(2pi*x)  *sin( pi*y)^2*sin(2pi*z)  *N[2]
    u[:,:,:,3] = @.  -sin(2pi*x)  *sin(2pi*y)  *sin( pi*z)^2*N[3]
    # u[:,:,:,1] .= 2
    # u[:,:,:,2] .= 1
    # u[:,:,:,3] .= 1
end

leVeque!(sim.flow.u,xList,yList,zList);

In [4]:
WaterLily.project!(sim.flow,sim.pois)
WaterLily.BCVecPerNeu!(sim.flow.u;Dirichlet=true, A=sim.flow.U, perdir=sim.flow.perdir)
save("aLeVequeubase.jld","data",sim.flow.u)

In [7]:
u⁰ = sim.flow.u
Statistics.mean
maximum(abs.(
    u⁰[3:end, 2:end-1, 2:end-1,1]-u⁰[2:end-1, 2:end-1, 2:end-1,1]+
    u⁰[2:end-1, 3:end, 2:end-1,2]-u⁰[2:end-1, 2:end-1, 2:end-1,2]+
    u⁰[2:end-1, 2:end-1, 3:end,3]-u⁰[2:end-1, 2:end-1, 2:end-1,3]
))/256

8.493662f-7

In [ ]:
GLMakie.contour(uMag,levels=[N[1]],alpha=1,isorange=N[1]/10)

In [ ]:
uu = copy(sim.flow.u);
leVeque!(sim.flow.u,xList,yList,zList);
uuMag = sqrt.(sum(uu.^2,dims=[4]))[:,:,:,1];

In [ ]:
GLMakie.contour(uuMag,levels=[N[1]],alpha=1,isorange=N[1]/10)


In [ ]:
uDiffMag = sqrt.(sum((sim.flow.u .- uu).^2,dims=[4]))[:,:,:,1];

In [ ]:
maximum(uDiffMag)